In [23]:
import nltk
print(nltk.data.path)


['C:\\Users\\Lenovo/nltk_data', 'C:\\Users\\Lenovo\\anaconda3\\nltk_data', 'C:\\Users\\Lenovo\\anaconda3\\share\\nltk_data', 'C:\\Users\\Lenovo\\anaconda3\\lib\\nltk_data', 'C:\\Users\\Lenovo\\AppData\\Roaming\\nltk_data', 'C:\\nltk_data', 'D:\\nltk_data', 'E:\\nltk_data']


In [25]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, SpatialDropout1D, Bidirectional
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [27]:
# Load Dataset
df = pd.read_csv("new_dataset.csv")

In [29]:
print(df.columns)

Index(['id', 'Question', 'Desired_answer', 'Student_answer', 'score_me',
       'score_other', 'score_avg'],
      dtype='object')


In [31]:
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)  # Remove special characters
    words = word_tokenize(text)  # Tokenization
    words = [word for word in words if word not in stopwords.words('english')]  # Remove stopwords
    return ' '.join(words)

In [39]:
import nltk
import os
import shutil
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Add your NLTK data path so nltk can find your downloaded resources
nltk_data_path = r'C:\Users\Lenovo\AppData\Roaming\nltk_data'
if nltk_data_path not in nltk.data.path:
    nltk.data.path.append(nltk_data_path)

# Download required NLTK packages (if not already downloaded)
nltk.download('punkt')
nltk.download('stopwords')

# Workaround: Create the 'punkt_tab' folder if it doesn't exist by copying 'punkt'
punkt_dir = os.path.join(nltk_data_path, 'tokenizers', 'punkt')
punkt_tab_dir = os.path.join(nltk_data_path, 'tokenizers', 'punkt_tab')

if not os.path.exists(punkt_tab_dir):
    if os.path.exists(punkt_dir):
        shutil.copytree(punkt_dir, punkt_tab_dir)
        print("Created 'punkt_tab' from 'punkt'.")
    else:
        print("The 'punkt' directory was not found in your nltk_data.")

# Define the text preprocessing function
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)  # Remove special characters
    words = word_tokenize(text)  # Tokenization
    words = [word for word in words if word not in stopwords.words('english')]  # Remove stopwords
    return ' '.join(words)

# Apply preprocessing to DataFrame columns
df['Question'] = df['Question'].apply(preprocess_text)
df['Desired_answer'] = df['Desired_answer'].apply(preprocess_text)
df['Student_answer'] = df['Student_answer'].apply(preprocess_text)

print("Preprocessing complete!")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\Lenovo/nltk_data'
    - 'C:\\Users\\Lenovo\\anaconda3\\nltk_data'
    - 'C:\\Users\\Lenovo\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\Lenovo\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\Lenovo\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [37]:
# Define tokenizer
MAX_VOCAB = 10000  # Max words to keep in vocabulary
MAX_LEN = 100  # Max length of input sequences
tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<OOV>")
tokenizer.fit_on_texts(df['Question'] + df['Desired_answer'] + df['Student_answer'])

AttributeError: 'float' object has no attribute 'lower'

In [ ]:
# Convert text to sequences
question_seq = tokenizer.texts_to_sequences(df['question'])
desired_seq = tokenizer.texts_to_sequences(df['desired_answer'])
student_seq = tokenizer.texts_to_sequences(df['student_answer'])

In [ ]:
# Pad sequences
question_seq = pad_sequences(question_seq, maxlen=MAX_LEN, padding='post')
desired_seq = pad_sequences(desired_seq, maxlen=MAX_LEN, padding='post')
student_seq = pad_sequences(student_seq, maxlen=MAX_LEN, padding='post')

In [ ]:
# Normalize output (score_avg)
scaler = MinMaxScaler()
df['score_avg'] = scaler.fit_transform(df[['score_avg']])

# Define input and output
X = np.concatenate([question_seq, desired_seq, student_seq], axis=1)
y = df['score_avg'].values

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Define LSTM Model
EMBEDDING_DIM = 128
model = Sequential([
    Embedding(MAX_VOCAB, EMBEDDING_DIM, input_length=X.shape[1]),
    SpatialDropout1D(0.2),
    Bidirectional(LSTM(128, return_sequences=True)),
    Dropout(0.3),
    Bidirectional(LSTM(64)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Sigmoid for normalized output
])

In [ ]:
# Compile model
model.compile(loss='mse', optimizer=Adam(learning_rate=0.001), metrics=['mae'])

# Train model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32)

# Evaluate model
loss, mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {mae}")

In [ ]:
# Function to Predict Grade
def predict_grade(question, desired_answer, student_answer):
    q_seq = pad_sequences(tokenizer.texts_to_sequences([preprocess_text(question)]), maxlen=MAX_LEN, padding='post')
    d_seq = pad_sequences(tokenizer.texts_to_sequences([preprocess_text(desired_answer)]), maxlen=MAX_LEN, padding='post')
    s_seq = pad_sequences(tokenizer.texts_to_sequences([preprocess_text(student_answer)]), maxlen=MAX_LEN, padding='post')
    input_seq = np.concatenate([q_seq, d_seq, s_seq], axis=1)
    predicted_score = model.predict(input_seq)[0][0]
    return scaler.inverse_transform([[predicted_score]])[0][0]

In [ ]:
# Example Prediction
print(predict_grade("What is AI?", "AI is a branch of computer science that deals with intelligence.", "AI is about machines learning to think."))